# Exploring indicators

This notebook explores the existence of Yi Peng's indicators in ABS and id_population sources.
Then we will try to segment by type and see if we can extrapolate the data.

## Function definitions and django manage

In [1]:
import pandas
import seaborn as sns
import matplotlib.pyplot as plt

import os
import pandas as pd

os.chdir('../deciml_django/')

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'deciml.settings')

from deciml.apps.db.models import *
from deciml.apps.common.utils import get_actuals

In [2]:
def sources_that_contain_indicator(indicator):
    """
    indicator should be the minimum you want to find. indicator='Hoc' would
    match both Hockey and ChOcOlate.
    """
    df = pd.DataFrame(Indicator.objects.filter(indicator_info__name__icontains=indicator)\
                               .values('indicator_info__id',
                                       'indicator_info__name', 
                                       'indicator_info__description',
                                       'source__id'))
    
    df = df.drop_duplicates()
    diff_sources = df.source__id.unique()
    diff_ind = df.indicator_info__name.unique()
    
    print('* Different indicators found ({ndi}): {di}'.format(ndi=len(diff_ind), di=diff_ind))
    print('* Different sources found ({nds}): {ds}'.format(nds=len(diff_sources), ds=diff_sources))
    return df

In [3]:
def indicators_from_source(source):
    df = pd.DataFrame(Indicator.objects.filter(source__id=source)\
                               .distinct('indicator_info__name')\
                               .values('indicator_info__name', 
                                       'indicator_info__description'))
    return df

In [4]:
def values_by_indicator_and_source(source, indicator):

    df = pd.DataFrame(

           Actual.objects.filter(indicator__source__id=source,
                                 indicator__indicator_info__name=indicator)\
                 .values('date', 'value', 'indicator__location__name')
    )
    
    return df


#### Target indicators

- [x] state population (ID_POPUL:Population, ABS:Population)
    * Poor time depth
- [x] city population (ID_POPUL:Population --> Description is wrong)
- [x] GDP (ABS:Gross domestic product)
    * Whole state
- [x] household income (ABS:Gross Disposable Income)
    * Whole state
- [ ] household indebtedness

#### Extra indicators

- [X] Unemployment Tate (ABS:unemployment rate)

## Exploring indicators and sources

We want to check if Yi Peng's indicators are available in several sources. Then we will check which sources have data about the vacancy/occupation, the cap_rate/yield or the specialty sales.

### Indicators from ID_POPUL and ABS

In [5]:
indicators_from_source('ID_POPUL')

,indicator_info__name,indicator_info__description
0,Average household size,Average people per household
1,Dwelling occupancy rate,Percentage of dwelling occupancy
2,Dwellings,Number of dwellings
3,Households,Number of households
4,Population,Number of persons living in a state
5,Population in non private dwellings,Population in non private dwellings


In [6]:
indicators_from_source('ABS')

,indicator_info__name,indicator_info__description
0,building approved,total value of approved service sector buildings
1,business gross,gross value of service sector businesses
2,business income,total value of income by service sector busine...
3,business lending,value lending to service sector businesses
4,Female to Male ratio,Female to male ratio of people living in a state
5,Gross Disposable Income,Household gross disposable income
6,Gross domestic product,Gross domestic product
7,Gross State Product,Gross State Product
8,Household Size Average,Average number of persons that live per household
9,Net Overseas Migration,Number of persons moving into/out of a country...


In [7]:
#Population by state
values_by_indicator_and_source('ABS', 'Population')

,date,value,indicator__location__name
0,2015-12-31,7616168.0,New South Wales Australia
1,2016-12-31,7732858.0,New South Wales Australia
2,2017-12-31,7867936.0,New South Wales Australia
3,2018-12-31,7988241.0,New South Wales Australia
4,2013-12-31,7404032.0,New South Wales Australia
5,2014-12-31,7508353.0,New South Wales Australia
6,2013-12-31,5772669.0,Victoria Australia
7,2014-12-31,5894917.0,Victoria Australia
8,2015-12-31,6022322.0,Victoria Australia
9,2016-12-31,6173172.0,Victoria Australia


In [8]:
#Population by city
values_by_indicator_and_source('ID_POPUL', 'Population').sort_values(by='date')

,date,value,indicator__location__name
2239,2008-06-30,30583.0,SOUTH BURNETT Australia
1062,2008-06-30,20020.0,GLENELG Australia
252,2008-06-30,89531.0,BALLARAT Australia
2283,2008-06-30,33451.0,SOUTHERN DOWNS Australia
1076,2008-06-30,44542.0,GLENORCHY Australia
...,...,...,...
856,2018-06-30,3259.0,DANDARAGAN Australia
867,2018-06-30,161609.0,DAREBIN Australia
2359,2018-06-30,17106.0,SUBIACO Australia
889,2018-06-30,6154.0,DENMARK Australia


In [9]:
#GDP by state??
values_by_indicator_and_source('ABS', 'Gross domestic product')

,date,value,indicator__location__name
0,1977-12-01,520030.0,Australia
1,1976-12-01,513066.0,Australia
2,1975-12-01,491687.0,Australia
3,1974-12-01,482157.0,Australia
4,1973-12-01,474983.0,Australia
5,1972-12-01,452270.0,Australia
6,1971-12-01,443965.0,Australia
7,1970-12-01,427620.0,Australia
8,1969-12-01,400444.0,Australia
9,1968-12-01,375292.0,Australia


In [10]:
# Gross Disposable Income by state??
values_by_indicator_and_source('ABS', 'Gross Disposable Income')

,date,value,indicator__location__name
0,2002-06-01,1137030.0,Australia
1,2001-06-01,1062416.0,Australia
2,2000-06-01,984253.0,Australia
3,1999-06-01,925488.0,Australia
4,1998-06-01,883062.0,Australia
5,1997-06-01,849097.0,Australia
6,1996-06-01,803289.0,Australia
7,1995-06-01,752868.0,Australia
8,1994-06-01,708298.0,Australia
9,1993-06-01,679594.0,Australia


In [11]:
# Unemployment rate by state
values_by_indicator_and_source('ABS', 'unemployment rate')

,date,value,indicator__location__name
0,2014-12-01,6.100007,Australia
1,2014-11-01,6.297957,Australia
2,2014-10-01,6.381043,Australia
3,2014-09-01,6.225912,Australia
4,2014-08-01,6.091151,Australia
...,...,...,...
3460,1979-10-01,7.868502,Western Australia Australia
3461,1979-09-01,7.216421,Western Australia Australia
3462,1979-08-01,7.297816,Western Australia Australia
3463,1979-07-01,7.573855,Western Australia Australia


In [12]:
values_by_indicator_and_source('ABS', 'unemployment rate').indicator__location__name.unique()

array(['Australia', 'New South Wales Australia', 'Queensland Australia',
       'South Australia Australia', 'Tasmania Australia',
       'Victoria Australia', 'Western Australia Australia'], dtype=object)

### Who has vacancy data (Or occupation)?

In [23]:
_ = sources_that_contain_indicator('vac')

* Different indicators found (17): ['Total Vacancy Grade Premium' 'Vacancy Rate Grade Premium'
 'Total Vacancy' 'Vacancy Rate' 'Total Vacancy Grade A'
 'Vacancy Rate Grade A' 'Total Vacancy Grade B' 'Vacancy Rate Grade B'
 'Total Vacancy Grade C' 'Vacancy Rate Grade C' 'Total Vacancy Grade D'
 'Vacancy Rate Grade D' 'Total Vacancy Secondary Grade'
 'Vacancy Rate Secondary Grade' 'Total Vacancy Overall'
 'Vacancy Rate Overall' 'Specialty Shops Vacant']
* Different sources found (1): ['JLL']


In [24]:
_

,indicator_info__name,indicator_info__description,source__id
0,Total Vacancy Grade Premium,Total Vacancy Grade Premium,JLL
1,Vacancy Rate Grade Premium,Vacancy Rate Grade Premium,JLL
2,Total Vacancy,Total Vacancy,JLL
3,Vacancy Rate,Vacancy Rate,JLL
4,Total Vacancy Grade A,Total Vacancy Grade A,JLL
5,Vacancy Rate Grade A,Vacancy Rate Grade A,JLL
6,Total Vacancy Grade B,Total Vacancy Grade B,JLL
7,Vacancy Rate Grade B,Vacancy Rate Grade B,JLL
8,Total Vacancy Grade C,Total Vacancy Grade C,JLL
9,Vacancy Rate Grade C,Vacancy Rate Grade C,JLL


In [25]:
_ = sources_that_contain_indicator('occ')

* Different indicators found (11): ['occupation rate' 'specialty occupancy cost' 'Dwelling occupancy rate'
 'Occupied Stock Grade Premium' 'Occupied Stock' 'Occupied Stock Grade A'
 'Occupied Stock Grade B' 'Occupied Stock Grade C'
 'Occupied Stock Grade D' 'Occupied Stock Secondary Grade'
 'Occupied Stock Overall']
* Different sources found (5): ['GPT' 'ID_POPUL' 'STOCKLAND' 'JLL' 'VICINITY']


In [15]:
_

,indicator_info__name,indicator_info__description,source__id
0,occupation rate,occupation rate,GPT
13,specialty occupancy cost,costs related to occupying a space in specialt...,GPT
70,Dwelling occupancy rate,Percentage of dwelling occupancy,ID_POPUL
198,specialty occupancy cost,costs related to occupying a space in specialt...,STOCKLAND
246,Occupied Stock Grade Premium,Occupied Stock Grade Premium,JLL
247,Occupied Stock,Occupied Stock,JLL
248,Occupied Stock Grade A,Occupied Stock Grade A,JLL
249,Occupied Stock Grade B,Occupied Stock Grade B,JLL
250,Occupied Stock Grade C,Occupied Stock Grade C,JLL
251,Occupied Stock Grade D,Occupied Stock Grade D,JLL


### Who has specialty data?

In [16]:
_ = sources_that_contain_indicator('specialty')

* Different indicators found (9): ['specialty occupancy cost' 'specialty sales' 'Prime Rents Specialty Rent'
 'Prime Rents Average Specialty Rental Growth' 'Specialty Shops Vacant'
 'Super-Prime Rents Average Specialty Rental Growth'
 'Super-Prime Rents Specialty Rent' 'Specialty Rent'
 'Average Specialty Rental Growth']
* Different sources found (5): ['GPT' 'STOCKLAND' 'VICINITY' 'SCENTRE' 'JLL']


In [17]:
_

,indicator_info__name,indicator_info__description,source__id
0,specialty occupancy cost,costs related to occupying a space in specialt...,GPT
18,specialty occupancy cost,costs related to occupying a space in specialt...,STOCKLAND
66,specialty occupancy cost,costs related to occupying a space in specialt...,VICINITY
223,specialty sales,sales in specialty retail,GPT
241,specialty sales,sales in specialty retail,STOCKLAND
283,specialty sales,sales in specialty retail,SCENTRE
325,specialty sales,sales in specialty retail,VICINITY
482,Prime Rents Specialty Rent,,JLL
492,Prime Rents Average Specialty Rental Growth,,JLL
502,Specialty Shops Vacant,,JLL


### Who has cap_rate data  (also yield)?

In [32]:
_ = sources_that_contain_indicator('cap')

* Different indicators found (6): ['capitalisation rate' 'Capital Value' 'Capital Value Growth q-o-q'
 'Capital Value Growth y-o-y' 'Capital Value Indicator Prime Grade'
 'Capital Value Indicator']
* Different sources found (5): ['GPT' 'STOCKLAND' 'JLL' 'SCENTRE' 'VICINITY']


In [33]:
_

,indicator_info__id,indicator_info__name,indicator_info__description,source__id
0,CAP_RATE,capitalisation rate,net operating income of a property divided by ...,GPT
50,CAP_RATE,capitalisation rate,net operating income of a property divided by ...,STOCKLAND
158,CAPITAL_VAL,Capital Value,Capital Value,JLL
159,CAPITAL_VAL_GRTH_QOQ,Capital Value Growth q-o-q,Capital Value Growth q-o-q,JLL
160,CAPITAL_VAL_GRTH_YOY,Capital Value Growth y-o-y,Capital Value Growth y-o-y,JLL
161,CAPITAL_VAL_IND_PRIME,Capital Value Indicator Prime Grade,Capital Value Indicator Prime Grade,JLL
178,CAP_RATE,capitalisation rate,net operating income of a property divided by ...,SCENTRE
221,CAP_RATE,capitalisation rate,net operating income of a property divided by ...,VICINITY
380,Capital_Value_Indicator,Capital Value Indicator,,JLL


In [29]:
_ = sources_that_contain_indicator('yield')

* Different indicators found (10): ['Equivalent Yield Upper Prime Grade' 'Equivalent Yield Lower Prime Grade'
 'Equivalent Average Yield Prime Grade'
 'Equivalent Yield Upper Secondary Grade'
 'Equivalent Yield Lower Secondary Grade'
 'Equivalent Average Yield Secondary Grade' 'Lower Yield'
 'Mid-Point Yield' 'Upper Yield' 'Median Yield']
* Different sources found (1): ['JLL']


In [30]:
_

,indicator_info__name,indicator_info__description,source__id
0,Equivalent Yield Upper Prime Grade,Equivalent Yield Upper Prime Grade,JLL
5,Equivalent Yield Lower Prime Grade,Equivalent Yield Lower Prime Grade,JLL
10,Equivalent Average Yield Prime Grade,Equivalent Average Yield Prime Grade,JLL
15,Equivalent Yield Upper Secondary Grade,Equivalent Yield Upper Secondary Grade,JLL
20,Equivalent Yield Lower Secondary Grade,Equivalent Yield Lower Secondary Grade,JLL
25,Equivalent Average Yield Secondary Grade,Equivalent Average Yield Secondary Grade,JLL
30,Lower Yield,,JLL
84,Mid-Point Yield,,JLL
116,Upper Yield,,JLL
170,Median Yield,,JLL


## Extracting the relevant data

In [6]:
get_stactuals(cursor=None, where={'indicator__source_id': 'JLL'}, columns=['value', 'date', 'indicator_id', 'indicator__source_id'], use_django_orm=True)

,indicator_id,date,value_change,error_code,st_processed,af_processed,folder_name,update_date,value,indicator__source_id
0,3947,1986-09-23,NaN,0.0,False,True,Office,2019-12-15 14:05:39.648309+00:00,4875.652174,JLL
1,3947,1980-01-01,NaN,0.0,False,True,Office,2019-12-15 14:05:39.648309+00:00,1590.000000,JLL
2,3947,1980-01-02,NaN,0.0,False,True,Office,2019-12-15 14:05:39.648309+00:00,1592.197802,JLL
3,3947,1980-01-03,NaN,0.0,False,True,Office,2019-12-15 14:05:39.648309+00:00,1594.395604,JLL
4,3947,1980-01-04,NaN,0.0,False,True,Office,2019-12-15 14:05:39.648309+00:00,1596.593407,JLL
...,...,...,...,...,...,...,...,...,...,...
1200907,9950,2013-10-23,NaN,128.0,False,True,Retail,2020-01-03 09:59:16.021078+00:00,0.075000,JLL
1200908,9950,2013-10-24,NaN,128.0,False,True,Retail,2020-01-03 09:59:16.021078+00:00,0.075000,JLL
1200909,9950,2013-10-25,NaN,128.0,False,True,Retail,2020-01-03 09:59:16.021078+00:00,0.075000,JLL
1200910,9950,2013-10-26,NaN,128.0,False,True,Retail,2020-01-03 09:59:16.021078+00:00,0.075000,JLL


In [8]:
get_actuals??

Vicinity, stockland y GPT tienen también datos de housing y offices (aparte de retail), solo nos interesa retail. Mirar la variable folder_name.

llamar get_actuals para sacar las cosas de jll.

Segmentar por regional/sub_regional/cvd.